In [1]:
import pandas as pd
import gtfs_functions as gtfs
import folium
from zipfile import ZipFile
from shapely import geometry
pd.set_option('display.max_rows',150)

In [76]:
import gtfs_kit as gk
import geopandas as gpd

In [69]:
path = './gtfs_data/raw/2021-10-24.zip'
feed = gk.read_feed(path, dist_units='km')

In [70]:
feed.validate()

,type,message,table,rows
0,warning,Feed expired,calendar/calendar_dates,[]
1,warning,Route has no trips,routes,"[13, 16, 17]"
2,warning,Stop has no stop times,stops,"[13, 14, 163, 270, 295, 299, 399, 400, 401, 40..."
3,warning,Unrecognized column block_name,trips,[]


In [81]:
rids = feed.routes.route_id.loc[2:4]
# feed.map_routes(rids, include_stops=True)
rids2 = gpd.GeoSeries(data=rids, crs=4326)
rids2

2    10C
3    10G
4     13
Name: route_id, dtype: object

In [83]:
routes, stops, stop_times, trips, shapes = gtfs.import_gtfs('./gtfs_data/20220111.zip', busiest_date=False)

In [86]:
routes.head()

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,1,1,1,Alton Park (Out & In),NaN,3,NaN,FCB814,000000
1,10A,1,10A,Avondale,NaN,3,NaN,9AD2AE,000000
2,10C,1,10C,Campbell Street/Highway 58,NaN,3,NaN,9AD2AE,000000
3,10G,1,10G,10G,NaN,3,NaN,9AD2AE,000000
4,13,1,13,Rossville,NaN,3,NaN,00A54F,000000


In [209]:
trips

,trip_id,route_id,service_id,direction_id,shape_id
0,204888020,1,1,0,shp-1-04
1,204889020,1,1,0,shp-1-03
2,204890020,1,1,0,shp-1-03
3,204891020,1,1,0,shp-1-03
4,204892020,1,1,0,shp-1-03
...,...,...,...,...,...
707,206975020,33,1,1,shp-33-51
708,207272020,21,1,0,shp-21-02
709,207273020,21,1,1,shp-21-53
710,207274020,21,1,0,shp-21-01


In [14]:
sorted_stop_times = (stop_times.sort_values(['departure_time']).reset_index(drop=True))

In [24]:
sorted_stop_times[sorted_stop_times.trip_id == '206282020'].reset_index(drop=True)[['trip_id', 'arrival_time', 'departure_time', 'stop_sequence', 'geometry']].head()

,trip_id,arrival_time,departure_time,stop_sequence,geometry
0,206282020,37260.0,37260.0,1,POINT (-85.26866 35.05615)
1,206282020,37478.0,37478.0,2,POINT (-85.28099 35.05604)
2,206282020,37643.0,37643.0,3,POINT (-85.29940 35.05162)
3,206282020,37657.0,37657.0,4,POINT (-85.30135 35.05193)
4,206282020,37678.0,37678.0,5,POINT (-85.30246 35.05246)


In [16]:
trips_shape = trips.merge(shapes, on=['shape_id'])

In [17]:
trips_shape.sort_values('shape_id').shape_id.nunique()

64

In [59]:
stops_trip_list = [([point.xy[1][0], point.xy[0][0]]) for point in stop_times[stop_times['trip_id'] == '204888020'].geometry]
# .append( point.stop_sequence for point in stop_times[stop_times['trip_id'] == '204888020' ])

In [61]:
stop_times[stop_times['trip_id'] == '204888020'].sort_values(by=['arrival_time'])

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,...,stop_code,stop_name,stop_desc,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding,geometry
12,204888020,31860.0,31860.0,354,1,NaN,0,0,0.00,1,...,2942,SHOLAR + CARTA,NaN,NaN,NaN,NaN,NaN,NaN,2,POINT (-85.26866 35.05615)
11,204888020,32078.0,32078.0,505,2,NaN,0,0,1383.38,0,...,2321,RIVERSIDE + RIVER PARK,NaN,NaN,NaN,NaN,NaN,NaN,2,POINT (-85.28099 35.05604)
10,204888020,32243.0,32243.0,784,3,NaN,0,0,3231.25,0,...,2092,MABEL + BATTERY PLACE,NaN,NaN,NaN,NaN,NaN,NaN,2,POINT (-85.29940 35.05162)
9,204888020,32257.0,32257.0,283,4,NaN,0,0,3425.34,0,...,1139,3RD + HOUSTON,NaN,NaN,NaN,NaN,NaN,NaN,2,POINT (-85.30135 35.05193)
8,204888020,32278.0,32278.0,284,5,NaN,0,0,3561.91,0,...,1140,3RD + LINDSAY,NaN,NaN,NaN,NaN,NaN,NaN,2,POINT (-85.30246 35.05246)
7,204888020,32323.0,32323.0,285,6,NaN,0,0,3853.10,0,...,1131,3RD + HIGH,NaN,NaN,NaN,NaN,NaN,NaN,2,POINT (-85.30529 35.05347)
21,204888020,32342.0,32342.0,713,7,NaN,0,0,3973.84,0,...,1141,3RD + LOOKOUT,NaN,NaN,NaN,NaN,NaN,NaN,2,POINT (-85.30654 35.05358)
20,204888020,32357.0,32357.0,286,8,NaN,0,0,4050.98,0,...,1151,3RD + WALNUT,NaN,NaN,NaN,NaN,NaN,NaN,2,POINT (-85.30734 35.05357)
19,204888020,32373.0,32373.0,287,9,NaN,0,0,4150.00,0,...,1120,3RD + CHERRY,NaN,NaN,NaN,NaN,NaN,NaN,2,POINT (-85.30842 35.05355)
18,204888020,32400.0,32400.0,1351,10,NaN,0,0,4362.64,1,...,2184,MARKET + 4TH,NaN,NaN,NaN,NaN,NaN,NaN,2,POINT (-85.30963 35.05266)


In [32]:
# stop_times[stop_times['trip_id'] == '204888020']

In [101]:
import random

In [102]:
def get_color():
    hexadecimal = ["#"+''.join([random.choice('ABCDEF0123456789') for i in range(6)])]
    return hexadecimal[0]

In [198]:
stops

,stop_id,stop_code,stop_name,stop_desc,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding,geometry
0,12,2201,MARKET + FAMILY DOLLAR,NaN,NaN,NaN,NaN,NaN,NaN,2,POINT (-85.30953 35.05070)
1,17,2164,MARKET + 11TH,NaN,NaN,NaN,NaN,NaN,NaN,2,POINT (-85.30943 35.04425)
2,18,3074,MARKET + 12TH,NaN,NaN,NaN,NaN,NaN,NaN,2,POINT (-85.30907 35.04235)
3,19,2204,MARKET + COWART,NaN,NaN,NaN,NaN,NaN,NaN,2,POINT (-85.30839 35.04095)
4,21,2217,MARKET + MAIN,NaN,NaN,NaN,NaN,NaN,NaN,2,POINT (-85.30732 35.03638)
...,...,...,...,...,...,...,...,...,...,...,...
1112,2339,1873,Harper & Bell-0,NaN,NaN,NaN,NaN,NaN,NaN,2,POINT (-85.31300 35.06842)
1113,2340,1874,Ziegler & May-0,NaN,NaN,NaN,NaN,NaN,NaN,2,POINT (-85.31202 35.06778)
1114,2341,2802,Frazier & Woodland,NaN,NaN,NaN,NaN,NaN,NaN,2,POINT (-85.30763 35.06262)
1115,2342,2805,Cherokee & Spring,NaN,NaN,NaN,NaN,NaN,NaN,2,POINT (-85.31489 35.06702)


In [218]:
stops_trip_list

[[35.042365, -85.309017],
 [35.040981, -85.308304],
 [35.038581, -85.307481],
 [35.03711, -85.307382],
 [35.036378, -85.307337],
 [35.043473, -85.30934],
 [35.044253, -85.309359],
 [35.053467, -85.305293],
 [35.052462, -85.302459],
 [35.051933, -85.301354],
 [35.051616, -85.299396],
 [35.05604, -85.280994],
 [35.056149, -85.268664],
 [35.045384, -85.309404],
 [35.045987, -85.309422],
 [35.04812, -85.309487],
 [35.048829, -85.309511],
 [35.050702, -85.309564],
 [35.05266, -85.309629],
 [35.053551, -85.308418],
 [35.053574, -85.307337],
 [35.053582, -85.306543],
 [35.034613, -85.308169],
 [35.032755, -85.30862],
 [35.031787, -85.308823],
 [35.030804, -85.309016],
 [35.027529, -85.310594],
 [35.026729, -85.311002],
 [35.025897, -85.311432],
 [35.010685, -85.325789],
 [34.989036, -85.3195]]

In [219]:
m = folium.Map([35.0456,-85.3097],zoom_start=15, tooltip = 'This tooltip will appear on hover' )
# for i, shp in shapes.iterrows():
#     # print(shp)
#     folium.Choropleth(
#         shp.geometry,
#         # shapes[shapes.shape_id=='shp-1-04'],
#         line_weight=3,   
#         # background_color='grey',
#         tooltip='shp-1-04',
#         line_color=get_color(),
#         # line_color='blue',
#         legend_name='Routes'
#     ).add_to(m)
folium.Choropleth(
    # shp.geometry,
    shapes[shapes.shape_id=='shp-1-04'],
    line_weight=3,   
    # background_color='grey',
    tooltip='shp-1-04',
    # line_color=get_color(),
    line_color='blue',
    legend_name='Routes'
).add_to(m)

i = 0
# for i, coordinates in stops.iterrows():
#     m.add_child(folium.CircleMarker(location = [coordinates.geometry.y, coordinates.geometry.x],
#                             radius=0.1, color='red',  popup= "Stop name:"+coordinates.stop_name,
#                                   weight=5))
    
for coordinates in stops_trip_list:
    m.add_child(folium.CircleMarker(location = coordinates,
                            radius=2, color='red',
                                  weight=5))
m

In [34]:
shapes['distance']=shapes.geometry.length

In [35]:
shapes

,shape_id,geometry,distance
0,shp-1-03,"LINESTRING (-85.30972 35.05266, -85.30963 35.0...",0.119208
1,shp-1-04,"LINESTRING (-85.26871 35.05617, -85.26863 35.0...",0.165311
2,shp-1-65,"LINESTRING (-85.31948 34.98907, -85.31936 34.9...",0.185436
3,shp-1-66,"LINESTRING (-85.31948 34.98907, -85.31936 34.9...",0.123177
4,shp-10A-01,"LINESTRING (-85.30864 35.04167, -85.30864 35.0...",0.097553
5,shp-10A-04,"LINESTRING (-85.26871 35.05617, -85.26863 35.0...",0.147805
6,shp-10A-06,"LINESTRING (-85.25008 35.07193, -85.25015 35.0...",0.158783
7,shp-10A-51,"LINESTRING (-85.26770 35.05770, -85.26760 35.0...",0.115083
8,shp-10A-52,"LINESTRING (-85.25008 35.07193, -85.25015 35.0...",0.086009
9,shp-10C-08,"LINESTRING (-85.17998 35.09329, -85.17999 35.0...",0.649243


In [127]:
trips_ext = pd.read_csv('./gtfs_data/20220111/trips.txt')
stop_times_ext = pd.read_csv('./gtfs_data/20220111/stop_times.txt')
routes_ext = pd.read_csv('./gtfs_data/20220111/routes.txt')

In [163]:
stop_times_ext.sort_values(by=['trip_id']).sort_values('stop_sequence', ascending=False).drop_duplicates(['trip_id'])[['trip_id', 'departure_time']]

,trip_id,departure_time
42597,206310020,07:00:00
48990,206378070,15:05:00
48378,206372070,08:30:00
48684,206375070,16:45:00
40293,206287020,16:45:00
...,...,...
56794,206508020,06:00:00
56792,206507020,19:45:00
56790,206488020,06:00:00
56804,206563070,19:25:00


In [190]:
st2 = stop_times_ext.sort_values(by=['trip_id', 'stop_sequence']).drop_duplicates(['trip_id'])[['trip_id', 'arrival_time']].merge(stop_times_ext.sort_values(by=['trip_id']).sort_values('stop_sequence', ascending=False).drop_duplicates(['trip_id'])[['trip_id', 'departure_time']], on='trip_id').merge(trips_ext, on=['trip_id']).sort_values(['block_id'])[['trip_id', 'route_id', 'block_id', 'arrival_time', 'departure_time', 'shape_id', 'trip_headsign']].sort_values(['block_id','arrival_time']).head(20).rename(columns={'departure_time':'trip_end_time', 'arrival_time':'trip_start_time'}).reset_index(drop=True)
st2
# stop_times_ext.drop_duplicates(['trip_id'])

,trip_id,route_id,block_id,trip_start_time,trip_end_time,shape_id,trip_headsign
0,205192020,10G,102,05:05:00,05:40:00,shp-10G-54,DOWNTOWN
1,204920020,1,102,05:40:00,06:10:00,shp-1-03,ALTON PARK
2,204951020,1,102,06:10:00,06:45:00,shp-1-66,DOWNTOWN
3,205172020,10G,102,06:45:00,07:15:00,shp-10G-04,GLENWOOD
4,205078020,10A,102,07:15:00,07:45:00,shp-10A-52,DOWNTOWN
5,204915020,1,102,07:45:00,08:15:00,shp-1-03,ALTON PARK
6,204946020,1,102,08:15:00,08:50:00,shp-1-66,DOWNTOWN
7,205059020,10A,102,08:50:00,09:20:00,shp-10A-01,AVONDALE
8,205187020,10G,102,09:20:00,09:50:00,shp-10G-54,DOWNTOWN
9,204911020,1,102,09:50:00,10:20:00,shp-1-03,ALTON PARK


In [210]:
# trips_ext.sort_values(by=['block_id']).head(20)
trips_ext.drop(['trip_short_name'], axis=1)

,trip_id,route_id,service_id,trip_headsign,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed,block_name
0,204888020,1,1,ALTON PARK,0,1002,shp-1-04,2,2,107
1,204889020,1,1,ALTON PARK,0,2802,shp-1-03,2,2,155
2,204890020,1,1,ALTON PARK,0,3002,shp-1-03,2,2,156
3,204891020,1,1,ALTON PARK,0,2802,shp-1-03,2,2,155
4,204892020,1,1,ALTON PARK,0,3002,shp-1-03,2,2,156
...,...,...,...,...,...,...,...,...,...,...
707,206975020,33,1,SHUTTLE PARK SOUTH,1,3902,shp-33-51,2,2,3302
708,207272020,21,1,GOLDEN GATEWAY,0,2602,shp-21-02,2,2,153
709,207273020,21,1,DOWNTOWN,1,2602,shp-21-53,2,2,153
710,207274020,21,1,GOLDEN GATEWAY,0,102,shp-21-01,2,2,101
